In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2

# DATA IMPORTS 
import random
import h5py
import numpy as np
import glob

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# MY OWN CLASSES
from SquareLocator import *
Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
data_dir = r"C:\Users\fhacesga\OneDrive - University Of Houston\AAA_RECTDNN\data/"

# input_folder = f"../data/HistoricalFIRMS/"
start_folder   = r"D:\FloodChange\AAA_HistoricalDownload/"
input_folder   = r"D:\FloodChange\ZZZ_Squares/"
output_folder  = r"D:\FloodChange\ZZZZ_Rectangles/"

In [3]:
def sharpeningKernel(n_window=5):
    kernel = np.zeros((n_window, n_window))
    kernel[n_window // 2, :] = -1
    kernel[:, n_window // 2] = -1
    kernel[ n_window // 2,  n_window // 2] = -1 * np.sum(kernel)
    return kernel

In [7]:
def identifyMainRectangle(image):
    
    image = image.T
    
    # PREPROCESS IMAGE
    sharpened_image = cv2.filter2D(image, -1, sharpeningKernel(201))
    edges = cv2.Canny(sharpened_image, 100, 100)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # LOOP THROUGH IDENTIFIED CONTOURS, SIMPLIFY, AND FILTER BY NUMBER / AREA
    approx_list = list()
    gen_approx_list = list()
    area_ratios = list()

    for contour in contours:
        epsilon = 0.1 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)    
        if image.ndim == 2:
            image_size = image.size
        else:
            image_size = image[:, :, 0].size
        area_ratio = cv2.contourArea(contour) / image_size
        gen_approx_list.append(contour)
        
        if len(approx) == 4 and area_ratio > 0.25:
            approx_list.append(approx)
            area_ratios.append(area_ratio)
    rectangles_image = image.copy()
    print(gen_approx_list)
    cv2.drawContours(rectangles_image, gen_approx_list, 0, (0, 255, 0), 2)
    return rectangles_image, [0]
    # IF IS EMPTY, CONTINUE
    if len(area_ratios) == 0:
        return None

    # OUTPUT LARGEST 4-SIDED CONTOUR
    print(area_ratios)
    idx = np.argmax(np.array(area_ratios))
    rectangles_image = image.copy() * 0
    cv2.drawContours(rectangles_image, [approx_list[idx]], 0, (0, 255, 0), 2)
    return rectangles_image, approx_list[idx]
    
    
def identifyBiggestContour(image, initial_image=None):
        
    if initial_image is None:
        initial_image = image
          
    if initial_image.ndim == 3:
        initial_image = initial_image[:, :, 0]

    # PREPROCESS IMAGE
    sharpened_image = np.where(image > 150, 255, 0).astype(np.uint8)
    edges = cv2.Canny(sharpened_image, 1, 1)
    edges = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=1)
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # LOOP THROUGH IDENTIFIED CONTOURS, SIMPLIFY, AND FILTER BY NUMBER / AREA
    area_ratios = list()
    if image.ndim == 2:
        image_size = image.size
    else:
        image_size = image[:, :, 0].size

    for contour in contours:
        area_ratio = cv2.contourArea(contour) / image_size
        area_ratios.append(area_ratio)

    idx = np.argmax(np.array(area_ratios))
    rectangles_image = np.dstack((initial_image.copy(), initial_image.copy(), initial_image.copy()))
    cv2.drawContours(rectangles_image, contours, -1, (0, 0, 255), 10)
    cv2.drawContours(rectangles_image, [contours[idx]], -1, (0, 255, 0), 10)
    
    epsilon = 0.1 * cv2.arcLength(contours[idx], True)
    approx = cv2.approxPolyDP(contours[idx], epsilon, True)
    cv2.drawContours(rectangles_image, [approx], -1, (255, 0 , 0), 10)
    
    
    return rectangles_image, contours[idx]


In [8]:
images = glob.glob(input_folder+"*.tif")
images.append(glob.glob(input_folder+"*.png"))
images.append(glob.glob(input_folder+"*.jpg"))

In [9]:
outdict = {}

for filename in tqdm(images):
    # Open the image file
    filename     = os.path.basename(filename)
    start_path   = os.path.join(start_folder, filename)
    image_path   = os.path.join(input_folder, filename)
    output_path  = os.path.join(output_folder, filename)

    # INPUT IMAGE AND PREP
    image         = np.asarray(cv2.imread(image_path, 0))
    initial_image = np.asarray(cv2.imread(start_path, 0))
    
    output = identifyBiggestContour(image, initial_image=initial_image)
    
    if output is None:
        continue
    
    output_image, output_contour = output[0], output[1]
    
    outdict.update({filename : output_contour} )    
    cv2.imwrite(output_path[:-4] + ".png", output_image)
    

  0%|          | 0/486 [00:00<?, ?it/s]

TypeError: expected str, bytes or os.PathLike object, not list

In [13]:
outdict

{'480077.tif': array([[[2887,  154]],
 
        [[2421,  153]],
 
        [[2903,  174]],
 
        [[2419, 3774]]], dtype=int32),
 '4800770005.tif': array([[[ 6165,  9779]],
 
        [[ 5768, 10697]],
 
        [[ 5899, 10797]],
 
        [[ 6165, 10699]]], dtype=int32),
 '480077A.tif': array([[[ 206, 4367]],
 
        [[ 196, 4376]],
 
        [[ 197, 4392]],
 
        [[ 208, 4389]]], dtype=int32),
 '480287B.tif': array([[[ 713,   49]],
 
        [[ 209,  186]],
 
        [[ 199, 1980]],
 
        [[ 209,  186]]], dtype=int32),
 '480289.tif': array([[[2407, 1174]],
 
        [[2387, 1177]],
 
        [[2386, 1207]],
 
        [[2406, 1205]]], dtype=int32),
 '480289A.tif': array([[[2031, 2192]],
 
        [[2134, 4088]],
 
        [[1511, 4231]],
 
        [[2134, 4088]]], dtype=int32),
 '480290A.tif': array([[[ 223, 1360]],
 
        [[ 219, 1415]],
 
        [[ 228, 1419]],
 
        [[ 234, 1377]]], dtype=int32),
 '480291.tif': array([[[2039,    9]],
 
        [[2760,   64]],
 
 